In [16]:
import csv
import json
import math
from math import sqrt
from SPARQLWrapper import SPARQLWrapper, JSON
import requests
import pandas as pd

rec_mov = ""

def csvReader(path):
    with open(path) as csv_data:
        data = list(csv.reader(csv_data, delimiter=',', skipinitialspace=True))
    return data


#средняя оценка
def avg(data):
    rated = list(filter(lambda x: x != -1, data))
    avg=sum(rated)/len(rated)
    return avg

#упорядоченная метрика сходства
def similarity(userU, data):
    userU_ratings = data[userU]
    sim={}
    for userV in range(1,len(data)):
        if userV == userU:
            continue   
        userV_ratings = data[userV]
        UV, userU_norm, userV_norm = 0, 0, 0
        
        for i in range(len(userU_ratings)):
            if userU_ratings[i] == -1 or userV_ratings[i] == -1:
                continue
            #числитель
            UV += userU_ratings[i] * userV_ratings[i]
            #знаменатели
            userU_norm += userU_ratings[i] * userU_ratings[i]
            userV_norm += userV_ratings[i] * userV_ratings[i]
            sim[userV] = round(UV / (sqrt(userU_norm)*sqrt(userV_norm)), 3)
    sim = sorted(sim.items(), key=lambda x: x[1], reverse=True)
    return sim
    
#рассчет оценки
def getExpRating(userU, data, k):
    avgU = avg(data[userU])
    exp_ratings = {}
    userU_ratings = data[userU]
    sim_list=similarity(userU, data); 
    mov_names = csvReader("movie_names.csv")
    #для каждого фильма, который еще не посмотрел пользователь
    for mov in range(len(userU_ratings)):
        count, num, denom = 0,0,0
        if userU_ratings[mov] == -1:
            #берем первых k похожих
            for i in range(0,k):
                userV = sim_list[i] #пара (userId, sim)
                if data[userV[0]][mov] != -1:
                    simVU = userV[1]
                    rV = avg(data[userV[0]])
                    rVI = data[userV[0]][mov]
                    num += simVU * (rVI - rV) #числитель
                    denom += abs(simVU) #знаменатель
            rating = round(avgU + num/denom, 3)
            exp_ratings[str(mov) + ". " + str(mov_names[mov][1])] = rating
    return exp_ratings

"""
выбираем у нескольких похожих пользователей фильмы, которые они смотрели дома в выходные
затем считаем среднюю оценку
выбираем фильмы, которые пользователь еще не смотрел и находим среди них наивысший рейтинг
"""

def getWkndFilms(context_day, context_place, data, userID):
    sim_list = similarity(userID, data) # отсчитывает ID с 0
    #print(simList)
    films = {}
    for mov in range(len(data[userID])):
        ratings = []
        for i in range(0,k):
            userV = sim_list[i] #пара (userId, sim)
            if data[userV[0]-1][mov]!=-1 and context_place[userV[0]][mov+1] == "h" and context_day[userV[0]][mov+1] == "Sun" or context_day[userV[0]][mov+1] == "Sat":
                ratings.append(data[userV[0]-1][mov])
        if len(ratings) > 0:
            films[mov] = round(sum(map(int, ratings))/len(ratings), 3)
    return films

    

def getResult(context_day, context_place, data, userID):
    wknd_films = getWkndFilms(context_day, context_place, data, userID)
    
    exp_rating = getExpRating(userID, data, k)
    mov_names = csvReader("movie_names.csv")
    rec_movie = {}
    for mov in wknd_films:
        if data[userID][mov] == -1:
            rec_movie[str(mov) + ". " + str(mov_names[mov][1])] = exp_rating[str(mov) + ". " + str(mov_names[mov][1])] 
    #rec_movie = sorted(rec_movie.items(), key=lambda x: x[1], reverse=True)
    max_rating = 0
    global rec_mov 
    for key, value in rec_movie.items():
        if value >= max_rating:
            rec_mov = key
            rec_mov_rating = value
            
    result = { 
        "user": userID + 1,
        "1": exp_rating,
        "2": {rec_mov: rec_mov_rating}
    }
    return result


In [17]:
context_day = []
context_day = csvReader("context_day.csv")
context_place = []
context_place = csvReader("context_place.csv")
data = []
for _ in csvReader("data.csv")[1:]:
    data.append(list(map(int, _[1:])))

k = 4
variant = 31 
userID = variant - 1  

result = getResult(context_day, context_place, data, userID)
print(json.dumps(result, indent=4))

json_file = open("output.json",'w')
json_file.write(json.dumps(result, indent=4))
json_file.close()

{
    "user": 31,
    "1": {
        "12. Star Wars: Episode V - The Empire Strikes Back": 1.274,
        "16. Goodfellas": 2.499,
        "29. The Green Mile": 4.177
    },
    "2": {
        "29. The Green Mile": 4.177
    }
}


In [18]:
rec_mov = rec_mov[4:]
API_ENDPOINT = "https://www.wikidata.org/w/api.php"
params = {
    'action' : 'wbsearchentities',
    'format' : 'json',
    'language' : 'en',
    'search': rec_mov
}
res = requests.get(API_ENDPOINT, params = params)

film_q = res.json()['search'][0]['id']

In [19]:
sparql = SPARQLWrapper("https://query.wikidata.org/sparql")

sparql_query = """

SELECT ?actor ?actorLabel 
WHERE {
  wd:""" + film_q + """ wdt:P161 ?actor.
  ?actor wdt:P21 wd:Q6581072.
  MINUS
  {?actor wdt:P570 ?deathDate}
  
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
 }
 
"""

sparql.setQuery(sparql_query)

sparql.setReturnFormat(JSON)
results = sparql.query().convert()


In [20]:
results_df = pd.json_normalize(results['results']['bindings'])
results_df[['actor.value', 'actorLabel.value']].head()

,actor.value,actorLabel.value
0,http://www.wikidata.org/entity/Q229268,Patricia Clarkson
1,http://www.wikidata.org/entity/Q272952,Bonnie Hunt
2,http://www.wikidata.org/entity/Q433198,Paula Malcomson


In [21]:
with open('sparqlOutput.txt', 'w') as file:
        json.dump(results, file, indent=4)